# KNN (K近傍法)

KNN（K近傍法）はシンプルなコンセプトです。データセットの中から、K個の最も近いアイテムを見つけます。それらのアイテムに”投票”させることにより、新しいデータの属性を予測します。 

例として、MovieLensのデータを用いましょう。ある映画にジャンルと人気度で最も近い10個の映画を用いて、評点の予測を行いましょう。

最初に、全てのデータをpandasのDataFrameに読み込みます。

In [1]:
import pandas as pd

r_cols = ['user_id', 'movie_id', 'rating']
ratings = pd.read_csv('c:/DataScience/DataScience/ml-100k/u.data', sep='\t', names=r_cols, usecols=range(3))
ratings.head()


,user_id,movie_id,rating
0,0,50,5
1,0,172,5
2,0,133,1
3,196,242,3
4,186,302,3


movie_IDでグループ化しましょう。そして、評点の総数（各映画の人気度）と評点の平均値を計算しましょう。

In [2]:
import numpy as np

movieProperties = ratings.groupby('movie_id').agg({'rating': [np.size, np.mean]})
movieProperties.head()

rating          
           size      mean
movie_id                 
1           452  3.878319
2           131  3.206107
3            90  3.033333
4           209  3.550239
5            86  3.302326

評点の元のデータは、映画同士の距離を計算するための使い勝手がよくありません。従って、正規化された評点の数値を含む新しいDataFrameを作ります。0は誰も評点していないことを意味し、1はもっとも人気の高い映画の値です。

In [3]:
movieNumRatings = pd.DataFrame(movieProperties['rating']['size'])
movieNormalizedNumRatings = movieNumRatings.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
movieNormalizedNumRatings.head()

,size
movie_id,
1,0.773585
2,0.222985
3,0.152659
4,0.356775
5,0.145798


それでは、u.itemファイルからジャンルの情報を取得しましょう。 19個のフィールドがあり、それぞれが特定のジャンルに対応しています。0はそのジャンルではないことを意味し、1はそのジャンルであることを意味します。各映画は一つ以上のジャンルを持つことがあります。

関連する全てのデータは、ディクショナリmovieDictに格納します。各要素は映画の名前、ジャンルのリスト、正規化された人気度のスコア、平均の評点を含みます。

In [5]:
movieDict = {}
with open(r'c:/DataScience/DataScience/ml-100k/u.item') as f:
    temp = ''
    for line in f:
        fields = line.rstrip('\n').split('|')
        movieID = int(fields[0])
        name = fields[1]
        genres = fields[5:25]
        genres = map(int, genres)
        movieDict[movieID] = (name, genres, movieNormalizedNumRatings.loc[movieID].get('size'), movieProperties.loc[movieID].rating.get('mean'))


例えば、movie IDが1の, "Toy Story"があります。

In [6]:
movieDict[1]

('Toy Story (1995)',
 [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 0.77358490566037741,
 3.8783185840707963)

それでは、ジャンルがどれだけ似ているか、人気度がどれだけ似ているかで”距離”を計算するための関数を定義しましょう。この関数が機能することを確認するために、IDが2の映画とIDが4の映画の距離を計算してみましょう。

In [7]:
from scipy import spatial

def ComputeDistance(a, b):
    genresA = a[1]
    genresB = b[1]
    genreDistance = spatial.distance.cosine(genresA, genresB)
    popularityA = a[2]
    popularityB = b[2]
    popularityDistance = abs(popularityA - popularityB)
    return genreDistance + popularityDistance
    
ComputeDistance(movieDict[2], movieDict[4])



0.8004574042309891

距離が大きいほど、映画が似ていないことになります。2と4の映画の内容を見て、同じ点や相違点を確かめてみましょう。

In [8]:
print movieDict[2]
print movieDict[4]


('GoldenEye (1995)', [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], 0.22298456260720412, 3.2061068702290076)
('Get Shorty (1995)', [0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 0.35677530017152659, 3.5502392344497609)


ある映画（この場合はトイストーリー）と、データセット内の他の全ての映画との距離を計算します。そして、距離でソートし、K個の最も近い映画を取得します。

In [9]:
import operator

def getNeighbors(movieID, K):
    distances = []
    for movie in movieDict:
        if (movie != movieID):
            dist = ComputeDistance(movieDict[movieID], movieDict[movie])
            distances.append((movie, dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(K):
        neighbors.append(distances[x][0])
    return neighbors

K = 10
avgRating = 0
neighbors = getNeighbors(1, K)
for neighbor in neighbors:
    avgRating += movieDict[neighbor][3]
    print movieDict[neighbor][0] + " " + str(movieDict[neighbor][3])
    
avgRating /= float(K)

Liar Liar (1997) 3.15670103093
Aladdin (1992) 3.81278538813
Willy Wonka and the Chocolate Factory (1971) 3.63190184049
Monty Python and the Holy Grail (1974) 4.0664556962
Full Monty, The (1997) 3.92698412698
George of the Jungle (1997) 2.68518518519
Beavis and Butt-head Do America (1996) 2.78846153846
Birdcage, The (1996) 3.44368600683
Home Alone (1990) 3.08759124088
Aladdin and the King of Thieves (1996) 2.84615384615


10個の最近傍の映画の平均評点は以下の通りです。

In [10]:
avgRating

3.3445905900235564

これは、トイストーリーの実際の平均評点と比較していかがでしょうか？

In [11]:
movieDict[1]

('Toy Story (1995)',
 [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 0.77358490566037741,
 3.8783185840707963)

いい感じですね。


## アクティビティ

Kに10を設定しましたが、10というのは適当な値です。Kの値を変更すると、結果にどのような影響があるでしょうか？

距離を求めるロジックも適当なものです。ロジックを改善することは可能でしょうか？